<a href="https://colab.research.google.com/github/mhuckvale/pals0039/blob/master/Answers_6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![PALS0039 Logo](https://www.phon.ucl.ac.uk/courses/pals0039/images/pals0039logo.png)](https://www.phon.ucl.ac.uk/courses/pals0039/)

# Exercise 6.1 Answers


(a) Load the IMDB review data set.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

%tensorflow_version 2.x
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, Flatten, SimpleRNN, LSTM, GRU
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

(Xtrain_seq,ytrain_seq),(Xtest_seq,ytest_seq)=imdb.load_data(num_words=10000)

print(Xtrain_seq[:5])
print(ytrain_seq[:5])



(b) print some basic parameters

In [0]:
listlengths=[]
for sequence in Xtrain_seq:
  listlengths.append(len(sequence))
print("Longest list",max(listlengths))

wordcodes=[]
for sequence in Xtrain_seq:
  wordcodes.append(max(sequence))
print("Highest code",max(wordcodes))

dictionary=imdb.get_word_index();
print("the",dictionary['the'])
reverse_dictionary={0:"padding",1:"BOS",2:"UNK"}
for (key,value) in dictionary.items():
  reverse_dictionary[value+3]=key

print(list(map(reverse_dictionary.get, range(10))))

print("First review")
review=[]
for i in range(len(Xtrain_seq[0])):
  review.append(reverse_dictionary[Xtrain_seq[0][i]])
print(" ".join(review))
  

(c) load the Glove embeddings

In [0]:
df=pd.read_csv('https://www.phon.ucl.ac.uk/courses/pals0039/data/glove.6B.100d.zip',header=None)
df.rename(columns={0:"word"},inplace=True)
print("Read %d word embeddings of length %d" % (len(df),len(df.columns)-1))
embedsize=len(df.columns)-1
df.head()

(d) build a look up index

In [0]:
glove_index={}
for i,word in enumerate(df.word):
  glove_index[word]=i

print("#words",len(glove_index))
print(glove_index['the'],glove_index['white'],glove_index['cat'])
print(df.word[glove_index['the']],df.word[glove_index['white']],df.word[glove_index['cat']])

glove_embed=np.array(df.iloc[:,1:])

(e) map IMDB vocabulary to glove dictionary

In [0]:
word_embed=np.zeros((10000,embedsize))
for i in range(10000):
  if i <= 3:
    word='.'
  elif i in reverse_dictionary:
    word=reverse_dictionary[i]
  else:
    print("Word index %d not found" % (i))
  if word.lower() in glove_index:
    eidx=glove_index[word.lower()]
  else:
    eidx=glove_index['.']
  if i < 10:
    print(i,word,eidx)
  word_embed[i,:]=glove_embed[eidx,:]

idx_the=glove_index['the']
emb_the=glove_embed[idx_the]
widx_the=dictionary['the']+3
wemb_the=word_embed[widx_the]
print(idx_the,emb_the)
print(widx_the,wemb_the);

(e) prepare sequences to fixed length padded at start

In [0]:
maxseq=500
Xtrain_pad=pad_sequences(Xtrain_seq,maxlen=maxseq,padding='pre',value=0)
Xtest_pad=pad_sequences(Xtest_seq,maxlen=maxseq,padding='pre',value=0)
print(Xtrain_pad.shape)
print(Xtrain_pad[:10,:25])

(f) build a model with embeddings in first layer

In [0]:
model = Sequential()
model.add(Embedding(10000, embedsize, weights=[word_embed], input_length=maxseq, trainable=False))
#model.add(SimpleRNN(32,return_sequences=True))
#model.add(GRU(32,return_sequences=True))
model.add(LSTM(32,return_sequences=False,dropout=0.2))
#model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())


(g) Train the model

In [0]:
Xval = Xtest_pad[:1000]
yval = ytest_seq[:1000]
# fit the model
history=model.fit(Xtrain_pad, ytrain_seq, epochs=10, verbose=1,batch_size=64,validation_data=(Xval,yval))
print(history.history.keys())

(h) Plot training graphs

In [0]:
hist=history.history
epochs=range(1,len(hist['loss'])+1)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(epochs,hist['loss'],'bo',label="Training loss")
plt.plot(epochs,hist['val_loss'],'b-',label="Validation loss")
plt.title("Training and Validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs,hist['accuracy'],'bo',label="Training accuracy")
plt.plot(epochs,hist['val_accuracy'],'b-',label="Validation accuracy")
plt.title("Training and Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

(i) Get test performance

In [0]:
score, acc = model.evaluate(Xtest_pad, ytest_seq,verbose=0)
print('Test loss:', score)
print('Test accuracy:', acc)

ypred=model.predict(Xtest_pad)
print(ypred.shape)
ypred=ypred.flatten()
print(ypred.shape)
print(list(zip(ytest_seq[:10],ypred[:10])))
